In [2]:
"""importing the necessary libraries, numpy for handling the mathematical complexities
cv2 - OpenCV, for contouring the image and classifying the data points based on colour differences"""

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#loading the test image to an image variable
#this part of the code will be changed dynamically to accept images loaded on the RPi

img = cv2.imread("/home/siddhant/Pictures/leaf.jpg", cv2.IMREAD_COLOR)

In [4]:
#function to display the image since the process is tedious with multiple lines of code for compatibility in Jupyter Notebooks
# the kernel will stop working if you simply close an image, you have to press a random key on the keyboard to successfully close the image file
# opened after this function is executed

def show(img, title="img"):
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [5]:
# converting image to gray scale for easy colour segregation
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# blur the image to reduce noise as it makes the contouring more accurate. 
blur = cv2.blur(gray, (3, 3))

# binary thresholding of the image. Changes the image into all black or all white and no gray-spots in between.
# colours closer to black turn complete black and likewise with white

THRESH_VAL = 220 # variable
ret, thresh = cv2.threshold(blur, THRESH_VAL, 255, cv2.THRESH_BINARY)

show(thresh, title="Thresholded Image")

In [6]:
# find contours
# finds a set of joined points around the black and white image borders. Helps in segmentation and auto-selection of parts of the image

contours, hierarchy = cv2.findContours(thresh, 
                                       cv2.RETR_TREE, 
                                        cv2.CHAIN_APPROX_SIMPLE)

In [7]:
# number of contour arrays, there are 17 arrays in this particular test image

print("Length of contours: {}".format(len(contours)))

Length of contours: 17


In [8]:
# here we use numpy to draw an image. This is a part I don't quite understand myself enough to document correctly, but it creates a bitmap image.
# this created image is later filled with the contours we stored in the variable earlier to draw a blueprint of the image parts we're interested in

drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
color_contours = (0, 255, 0) # color for contours
for i in range(len(contours)):
    cv2.drawContours(drawing, contours, i, color_contours, 2, 8, hierarchy)

In [9]:
show(drawing, title='Contours')

In [10]:
# The important part, here we run two simple loops to find the maximum and minimum distance from the length and the width axes respectively
# The max and min are recorded after being compared to every single element of the contour array

max_x = 0
max_y = 0
min_x = thresh.shape[1]
min_y = thresh.shape[0]

for i in range(len(contours)):
    # print("Finding for: ", contours[i])
    for j in contours[i]:
        if(j[0][0] > max_x and j[0][0] < 1250):
            max_x = j[0][0]
        if(j[0][1] > max_y and j[0][1] < 850):
            max_y = j[0][1]
        if(j[0][0] < min_x and j[0][0] > 10):
            min_x = j[0][0]
        if(j[0][1] < min_y and j[0][1] > 10):
            min_y = j[0][1]
            
print("Finally: max_x = {}, max_y = {}".format(max_x, max_y))
print("Finally: min_x = {}, min_y = {}".format(min_x, min_y))

Finally: max_x = 1220, max_y = 766
Finally: min_x = 48, min_y = 92


In [11]:
# Naturally the length and width of the leaf are the difference between the points of min and max.

print(f'Length {max_x - min_x}')
print("Width: {}".format(max_y - min_y))

# These values will be sent to a code further to calculate actual value in metric units.

Length 1172
Width: 674
